В силу ограниченного времени я реализую функции подразумевая, что они будут применяться условного к файлам с кодом. В файлах с кодом нет длинных строк (PEP8 не длиннеe 79 символов)

Если эти функции применять к файлам с расширением, скажем, png, то код будет не эффективен.

In [93]:
import string
from time import time
import pickle as pck

Создадим отдельный класс для хранения строк отличия.
В нем будем хранить строку, если она удалялась.
Или координаты строк которые были добавлены.

In [94]:
class DiffString:
    def __init__(self,
                 begin_old: int, end_old: int,
                 begin_new: int, end_new: int,
                 deleted_str: string = ''):
        """Constructor"""
        self.begin_old = begin_old
        self.end_old = end_old
        self.begin_new = begin_new
        self.end_new = end_new
        self.deleted_str = deleted_str

Функция создает список объектов DiffString, в которых хранятся строки отличия между файдам old_file  и new_file


In [95]:
def GetDiffList(old_file, new_file):
    old_file_lines = [line for line in old_file]
    new_file_lines = [line for line in new_file]
    result = []

    last_checked_str = 0
    for k in range(len(old_file_lines)):

        next_bidder = -1
        go_next = True

        while go_next and (last_checked_str + next_bidder + 1 < len(new_file_lines)):
            next_bidder += 1
            if old_file_lines[k] == new_file_lines[last_checked_str + next_bidder]:
                go_next = False

        if go_next:
            result.append(DiffString(k, k, -1, -1, old_file_lines[k]))
        elif next_bidder == 0:
            last_checked_str += 1
        else:
            result.append(DiffString(-1, -1, last_checked_str, last_checked_str + next_bidder))
            last_checked_str = last_checked_str + next_bidder + 1

    return result

Публичная функция, которая создает diff файл с именем 'old_to_new_time'.
Принимает на вход имена двух файлов, находящихся в той же директории, что и файл кода. Возращает имя diff файла.

In [96]:
def CalculateTheDiff(old: string, new: string):
    old_file = open(old, 'rb')
    new_file = open(new, 'rb')

    diff_file_name = old + '_to_' + new + '_' + str(time())
    diff_file = open(diff_file_name, 'wb')

    list_of_diff = GetDiffList(old_file, new_file)
    pck.dump(list_of_diff, diff_file)

    old_file.close()
    new_file.close()
    diff_file.close()

    return diff_file_name

Восстанавливает из "старого" файла новый по дифф файлу

In [97]:
def Restore(old_file: string, diff_file: string):
    old_file_fd = open(old_file, 'rb')
    old_file_lines = [line for line in old_file_fd]
    diff_file_fd = open(diff_file, 'rb')
    differences = pck.load(diff_file_fd)
    diff_file_fd.close()

    shift = 0

    for diff_line in differences:
        if diff_line.begin_old == -1:
            for i in range(diff_line.begin_new, diff_line.end_new):
                old_file_lines.pop(diff_line.begin_new + shift)
            shift -= (diff_line.end_new - diff_line.begin_new)
        else:
            old_file_lines.insert(diff_line.begin_old, diff_line.deleted_str)
            shift += 1

    old_file_fd.close()

    with open(old_file, 'wb') as answer:
        for line in old_file_lines:
            answer.write(line)

Пример выполнения:


In [98]:
print('ИСХОДНЫЙ ФАЙЛ')
print('-'*30)

with open('old', 'rb') as f:
  for line in f:
    print(line)
print('_'*30)

print('ИЗМЕНЕННЫЙ ФАЙЛ')
print('-'*30)
with open('new', 'rb') as f:
  for line in f:
    print(line)

ИСХОДНЫЙ ФАЙЛ
------------------------------
b'import string\n'
b'import abc\n'
b'from collections import defaultdict\n'
b'\n'
b'from main.config import ALPHABET_POWER, ALPHABET_LETTER_CODE\n'
b'from main.encode import CaesarEncoderDecoder\n'
b'from main.train import DefaultTrainer\n'
b'\n'
b'\n'
b'class Hacker:\n'
b'    __metaclass__ = abc.ABCMeta\n'
b'\n'
b'    def __init__(self, model):\n'
b'        self.model = defaultdict(float, model)\n'
b'\n'
b'    @abc.abstractmethod\n'
b'    def hack(self, text: str, subcomm=None):\n'
b'        pass\n'
b'\n'
b'\n'
b'class CaesarHacker(Hacker):\n'
b'\n'
b'    def __init__(self, model):\n'
b'        super().__init__(model)\n'
b'        self.caesar_decoders = [CaesarEncoderDecoder(shift) for shift in range(ALPHABET_POWER)]\n'
b'        self.trainer = DefaultTrainer()\n'
b'\n'
b'    def hack(self, text: str, subcomm=None):\n'
b'        results = [0 for shift in range(ALPHABET_POWER)]\n'
b'        shift_result = 0\n'
b'\n'
b'        self.trainer.fe

In [99]:
name = CalculateTheDiff('old', 'new')
Restore('new', name)

In [100]:
print('ИСХОДНЫЙ ФАЙЛ')
print('-'*30)

with open('old', 'rb') as f:
  for line in f:
    print(line)
print('_'*30)

print('ИЗМЕНЕННЫЙ ФАЙЛ')
print('-'*30)
with open('new', 'rb') as f:
  for line in f:
    print(line)

ИСХОДНЫЙ ФАЙЛ
------------------------------
b'import string\n'
b'import abc\n'
b'from collections import defaultdict\n'
b'\n'
b'from main.config import ALPHABET_POWER, ALPHABET_LETTER_CODE\n'
b'from main.encode import CaesarEncoderDecoder\n'
b'from main.train import DefaultTrainer\n'
b'\n'
b'\n'
b'class Hacker:\n'
b'    __metaclass__ = abc.ABCMeta\n'
b'\n'
b'    def __init__(self, model):\n'
b'        self.model = defaultdict(float, model)\n'
b'\n'
b'    @abc.abstractmethod\n'
b'    def hack(self, text: str, subcomm=None):\n'
b'        pass\n'
b'\n'
b'\n'
b'class CaesarHacker(Hacker):\n'
b'\n'
b'    def __init__(self, model):\n'
b'        super().__init__(model)\n'
b'        self.caesar_decoders = [CaesarEncoderDecoder(shift) for shift in range(ALPHABET_POWER)]\n'
b'        self.trainer = DefaultTrainer()\n'
b'\n'
b'    def hack(self, text: str, subcomm=None):\n'
b'        results = [0 for shift in range(ALPHABET_POWER)]\n'
b'        shift_result = 0\n'
b'\n'
b'        self.trainer.fe

Что бы я добавил:

Можно обернуть все в класс, чтобы защитить приватные методы и внутренний класс.
Поменял бы класс для диф_строк и оптимизировал алгоритм.

Почистил код стайл :)